# biorbd #1 - Rototrans

This tutorial is a trivial example on how to manage Rototranslation object with biorbd. It is designed to show how one can find the rototranslation matrix between any segment.
The rototranslation $^{j}T_i$ (aka homogeneous transform) allows to transform any point $^{i}\boldsymbol{p}$ in frame $i$ to another frame $j$. It is defined as follow:
\begin{equation}
^{j}T_i = \begin{pmatrix}R_{11}&R_{12}&R_{13}&t_{x}\\ 
R_{21}&R_{22}&R_{23}&t_{y}\\ 
R_{31}&R_{32}&R_{33}&t_{z}\\ 
0&0&0&1\\
\end{pmatrix}
\end{equation}
Terms $R_{ij}$ come from the rotational part of homogenous transform, also known as Rotation matrix which would be in this case: $^{j}R_i$. And $t$ terms come from the translation part of homogenous transform which would be in this case: $^{i}\boldsymbol{t}$ the translation in frame $i$. 

In our example, we will use the musculoskeletal model illustrated just below.

<p style="text-align: center;">
<img src="../doc/wu_model_pic.png" width="400" height="400" /></p>

### Mathematical refreshers

Let $ ^{w}T_{s} $ the homogeneous transform matrix between a frame of any segment $s$ and world's frame $w$. We have :

$^{w}\boldsymbol{p}$=$^{w}T_{s} \; ^{s}\boldsymbol{p} $

If we want the rototrans matrix between two segment $i$ and $j$, we need to pass trough the world's frame $w$ :

$ ^{i}T_{j}$ =$ ^{i}T_{w} \;  ^{w}T_{j} $

with $ ^{i}T_{w}$ = $ ^{w}T_{i}^{-1}$

To sum up, the position $\boldsymbol{p}$ in frame $i$ in relation to $j$ is given by the rototranslation matrix $^{i}T_{j}$, multiplied by the position $\boldsymbol{p}$ in $j$ such as: $ ^{i}\boldsymbol{p}$ =$  ^{i}T_{j} \; ^{j}\boldsymbol{p}$ 
___

In [8]:
import bioviz
import numpy as np
import biorbd

In [22]:
# We load the model
model_name = "../models/wu_converted_definitif.bioMod"
# giving the path
biorbd_model = biorbd.Model(model_name)
# Now the model is loaded as a biorbd object
print(biorbd_model.nbQ())

17
thorax_parent_offset
thorax_translation
thorax_rotation_transform
thorax_reset_axis
thorax
clavicle_parent_offset
clavicle_translation
clavicle_sternoclavicular_r1
clavicle_sternoclavicular_r2
clavicle_sternoclavicular_r3
clavicle_reset_axis
clavicle
scapula_parent_offset
scapula_translation
scapula_rotation_transform
scapula_reset_axis
scapula
humerus_parent_offset
humerus_translation
humerus_shoulder_plane
humerus_shoulder_ele
humerus_shoulder_rotation
humerus_reset_axis
humerus
ulna_parent_offset
ulna_translation
ulna_rotation1
ulna_rotation2
ulna_elbow_flexion
ulna_reset_axis
ulna
radius_parent_offset
radius_translation
radius_rotation1
radius_pro_sup
radius_rotation3
radius_reset_axis
radius
hand_r_parent_offset
hand_r


In this example we will try to find the Rotranslation matrix between ulna, denoted $u$ and humerus, denoted $h$ when the elbow is bent. We set the degrees of freedom of the model in a chosen pose for the sake of the tutorial.

In [26]:
# Q is supposed to be same dimension as biorbd_model.nbQ(). here is the Q when the elbow is bend (rotation of ulna by pi/2)
Q = np.array(
    [ 
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        1.6282,
        0.0,
        1.1806,
        0.0
    ]
) 
# display the list of segment in the model
print("Here is the list of segment in the model: ")
for ii in range(biorbd_model.nbSegment()):
    print(biorbd_model.segment(ii).name().to_string())

Here is the list of segment in the model
thorax_parent_offset
thorax_translation
thorax_rotation_transform
thorax_reset_axis
thorax
clavicle_parent_offset
clavicle_translation
clavicle_sternoclavicular_r1
clavicle_sternoclavicular_r2
clavicle_sternoclavicular_r3
clavicle_reset_axis
clavicle
scapula_parent_offset
scapula_translation
scapula_rotation_transform
scapula_reset_axis
scapula
humerus_parent_offset
humerus_translation
humerus_shoulder_plane
humerus_shoulder_ele
humerus_shoulder_rotation
humerus_reset_axis
humerus
ulna_parent_offset
ulna_translation
ulna_rotation1
ulna_rotation2
ulna_elbow_flexion
ulna_reset_axis
ulna
radius_parent_offset
radius_translation
radius_rotation1
radius_pro_sup
radius_rotation3
radius_reset_axis
radius
hand_r_parent_offset
hand_r


We will consider the two segments ulna $u$ and humerus $h$, in order to build the rototranslation matrices between them. Let's find their index.

In [28]:
id_humerus = biorbd_model.getBodyBiorbdId("humerus") # this function can be use to find a segment name
id_ulna = biorbd_model.getBodyBiorbdId("ulna")
print("Humerus id: ", id_humerus)
print("Ulna id: ", id_ulna)

Humerus id:  23
Ulna id:  30


Now let's build the rototrans (aka homogeneous transform) of ulna and the humerus from local frame to world's frame.


In [29]:
Rototrans_matrix_world_humerus = biorbd_model.globalJCS(Q, id_humerus).to_array() # We converted the matrix into array in order to do matrice's operation thanks to numpy
Rototrans_matrix_world_ulna = biorbd_model.globalJCS(Q, id_ulna).to_array() # globaJCS() return the joint coordinate system (JCS) for the segment in global reference frame at a given Q.
print("T_world_humerus")
print(Rototrans_matrix_world_humerus)
print("")
print("T_world_ulna")
print(Rototrans_matrix_world_ulna)

T_world_humerus
[[ 9.99999957e-01 -2.07816773e-04  2.03543983e-04 -2.17852518e-02]
 [-2.15133279e-04 -5.73721101e-02  9.98352839e-01 -2.72586523e-03]
 [-1.95797038e-04 -9.98352843e-01 -5.73721534e-02  2.16064773e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

T_world_ulna
[[ 0.38168936 -0.9220298   0.06460863 -0.01520385]
 [-0.05639745  0.04653795  0.99732319  0.00449058]
 [-0.92256846 -0.38431141 -0.03423709  0.52651077]
 [ 0.          0.          0.          1.        ]]


We want to find the transformation matrix between humerus $h$ and ulna $u$
$^{h}T_{u}$=$^{h}T_{w}. $$^{w}T_{u}$

$^{h}T_{w}$ is the inverse matrix of ${^{w}T_{h}}^{-1}$. It is obtained using .transpose() method on the Rototrans object which actually inverses the rototranslation matrix.

In [30]:
Rototrans_matrix_humerus_world = biorbd_model.globalJCS(Q, id_humerus).transpose().to_array()
# Be aware that Rototrans_matrix_world_humerus.transpose() would give the wrong matrix

Rototrans_matrix_humerus_ulna = np.matmul(Rototrans_matrix_humerus_world, Rototrans_matrix_world_ulna)

print("T_humerus_ulna")
print(Rototrans_matrix_humerus_ulna)

NameError: name 'nb_humerus' is not defined

We now have $^{h}T_{u}$.

So, we are now able to transform any point in ulna's frame to humerus' frame with the following formula:

$ ^{h}\boldsymbol{p} =  ^{h}T_{u} \; ^{u}\boldsymbol{p}$ 

## Do It Yourself 

Now that you saw an example, let's apply what you learn on an other example. You have to find the rototrans matrix between humerus and scapula.

In [ ]:
idx_scapula = biorbd_model.GetBodyBiorbdId(...) # this function can be use to find a segment index using the segment name

In [ ]:
# Complete the following line 

Rototrans_matrix_world_scapula = biorbd_model.globalJCS(Q, ...).to_array() # to complete, find the number which correspond to the scapula
Rototrans_matrix_world_humerus = biorbd_model.globalJCS(Q, ...).to_array() # to complete, find the number which correspond to the humerus

Rototrans_matrix_scapula_world = biorbd_model.globalJCS(Q, ...). ...().to_array() # to complete, find the right function to inverse matrix

# Finally
Rototrans_matrix_scapula_humerus = np.matmul(... , ... ) #put the right matrix in the matrix multiplication function

print(Rototrans_matrix_scapula_humerus)


# Congratulation! You went from zero to hero in the understanding of rototrans matrices !